In [28]:
print("Start")

import pandas as pd

data = pd.read_csv("../../../dataset-sis-energy-derived-reanalysis-temperature-country_daily/H_ERA5_ECMW_T639_TA-_0002m_Euro_NUT2_S197901010000_E202309302300_INS_TIM_01d_subcountry_2023_10_15.csv", sep=";")

Start


In [29]:
# data.head()
# data.info()

In [30]:
df = data[["Date", "ES30"]].copy()
df

,Date,ES30
0,1979-01-01,281.061314
1,1979-01-02,279.750892
2,1979-01-03,279.663439
3,1979-01-04,278.837795
4,1979-01-05,277.102388
...,...,...
16339,2023-09-26,292.580450
16340,2023-09-27,292.713026
16341,2023-09-28,293.107935
16342,2023-09-29,294.105669


In [31]:
# convert column holding the date from Object to Datetime
df["Date"] = pd.to_datetime(df["Date"])

# resample from daily to week data and aggregate on mean temperature values over the week
# not resample("7D") as before as overflow to following year happens
# df = df.resample("1W", on="Date").max()

# convert temperature values from Kelvon to Celsius
df["ES30"] = df["ES30"] - 273.15

In [32]:
# reset index from date value to integer 
df = df.reset_index()
# get weeknumber as separate column
df["weeknumber"] = df.Date.dt.isocalendar().week
# get year number as separate column
df["year"] = df.Date.dt.isocalendar().year

In [33]:
# view data
# df

In [34]:
# set to "hot", when temp higher than 20 degrees
compareToInCelsius = 20
df["ES30_masked"] = df.ES30.mask(df.ES30 > compareToInCelsius, "hot")

In [35]:
grouped = df.groupby(["year", "weeknumber"])
grouped.describe()

index                                                 \
                count     mean       std      min       25%      50%   
year weeknumber                                                        
1979 1            7.0      3.0  2.160247      0.0      1.50      3.0   
     2            7.0     10.0  2.160247      7.0      8.50     10.0   
     3            7.0     17.0  2.160247     14.0     15.50     17.0   
     4            7.0     24.0  2.160247     21.0     22.50     24.0   
     5            7.0     31.0  2.160247     28.0     29.50     31.0   
...               ...      ...       ...      ...       ...      ...   
2023 35           7.0  16313.0  2.160247  16310.0  16311.50  16313.0   
     36           7.0  16320.0  2.160247  16317.0  16318.50  16320.0   
     37           7.0  16327.0  2.160247  16324.0  16325.50  16327.0   
     38           7.0  16334.0  2.160247  16331.0  16332.50  16334.0   
     39           6.0  16340.5  1.870829  16338.0  16339.25  16340.5   

                                    ES30                                  \
                      75%      max count       mean       std        min   
year weeknumber                                                            
1979 1               4.50      6.0   7.0   5.763790  1.415068   3.952388   
     2              11.50     13.0   7.0   6.800180  1.856289   4.477148   
     3              18.50     20.0   7.0   4.950309  1.973584   2.456835   
     4              25.50     27.0   7.0   7.274088  1.151949   5.499719   
     5              32.50     34.0   7.0   5.860998  2.992869   2.200394   
...                   ...      ...   ...        ...       ...        ...   
2023 35          16314.50  16316.0   7.0  20.091944  2.221103  17.289265   
     36          16321.50  16323.0   7.0  21.126405  1.419516  18.546076   
     37          16328.50  16330.0   7.0  18.780163  1.691075  15.770193   
     38          16335.50  16337.0   7.0  15.648390  1.439464  13.794638   
     39          16341.75  16343.0   6.0  19.993229  1.009813  18.672057   

                                                             
                       25%        50%        75%        max  
year weeknumber                                              
1979 1            4.840350   5.687795   6.557166   7.911314  
     2            5.555729   6.828763   7.847871   9.488150  
     3            4.187259   4.763351   5.072416   8.912625  
     4            6.568791   7.696574   7.879270   8.826197  
     5            3.840700   4.188361   8.924934   9.106963  
...                    ...        ...        ...        ...  
2023 35          18.675408  19.993416  21.211421  23.587265  
     36          20.710997  21.380067  21.901474  22.733753  
     37          18.047985  18.944682  19.983678  20.682938  
     38          14.437445  15.665216  17.025749  17.152491  
     39          19.463594  19.760480  20.706235  21.380236  

[2335 rows x 16 columns]

In [36]:
df_grouped = grouped["ES30"].max().to_frame(name="max_daily_in_week").reset_index()
df_grouped

,year,weeknumber,max_daily_in_week
0,1979,1,7.911314
1,1979,2,9.488150
2,1979,3,8.912625
3,1979,4,8.826197
4,1979,5,9.106963
...,...,...,...
2330,2023,35,23.587265
2331,2023,36,22.733753
2332,2023,37,20.682938
2333,2023,38,17.152491


In [37]:
# save result to a csv file
df_grouped.to_json('temp_subcountry_ES30_weekly_max_daily_in_week.json', orient='records')
